# Reconstruct phantom data
This exercise shows how to handle data from the Siemens mMR. It shows how to get from listmode data to sinograms, get a randoms estimate, and reconstruct using normalisation, randoms and attenuation.
(Scatter is not yet available from in SIRF).

It is recommended you complete the first part of `ML_reconstruct.ipynb` exercise first.

This exercise uses data from a phantom acquisition at UCL on a Siemens mMR. The phantom is the NEMA phantom (essentially a torso-shaped perspex box, with some spherical inserts). You will need to download that data. Please use the `SIRF-Exercises/scripts/download_PET_data.sh` script which will get the data, and make symbolic links in the location expected in this script. 

The script should work for other data of course, but you will need to adapt filenames.

Note that we currently don't show how to extract the data from the console. Please
[check our wiki for more information](https://github.com/CCPPETMR/SIRF/wiki/PET-raw-data).

Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: 8th of September 2016  
Second Version: 17th of May 2018

CCP PETMR Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2018 University College London.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from sirf.Utilities import show_2D_array, examples_data_path
from sirf.STIR import *

data_path = os.path.join('..', '..', 'data')
nema_data_path = os.path.join(data_path, 'PET', 'mMR', 'NEMA_IQ')
print('Finding files in %s' % nema_data_path)
os.chdir(nema_data_path)

In [ ]:
# check content of current directory using an iPython "magic" command
%ls

In [ ]:
# convert the Siemens Normalisation into STIR format, some keys slightly differ
!convertSiemensInterfileToSTIR.sh 20170809_NEMA_UCL.n.hdr norm.n.hdr
!convertSiemensInterfileToSTIR.sh 20170809_NEMA_MUMAP_UCL.v.hdr umap.v.hdr

# These files will sometimes have lines be terminated with CR instead of CRLF, fix this
# This just means, if we see CR<character> and <character> isn't LF, replace it with CRLF<character>
!sed -i "s/\r\([^\n]\)/\r\n\1/g" norm.n.hdr
!sed -i "s/\r\([^\n]\)/\r\n\1/g" umap.v.hdr

In [ ]:
#%% set filenames 
# input files
list_file = '20170809_NEMA_60min_UCL.l.hdr'
norm_file = 'norm.n.hdr'
attn_file = 'umap.v.hdr'
# output filename prefixes
sino_file = 'sino'

In [ ]:
# redirect STIR messages to some files
# you can check these if things go wrong
msg_red = MessageRedirector('info.txt', 'warn.txt')

# Creating sinograms from listmode data
Modern PET scanners can store data in listmode format. This is essentially a long list of all events detected by the scanner. We are interested here in the *prompts* (the coincidence events) and the *delayed events* (which form an estimate of the *accidental coincidences* in the prompts.

We show how to histogram the prompts into a sinogram etc.

## First create a template for the sinogram
This template is used to specify the sizes of the output sinogram.

It is often the case in PET that we use sinograms with "larger" bins, i.e. combine data from several detector pairs into a single bin. This reduces size of the final sinogram, and decreases computation time. The terminology here is somewhat complicated, but *span* uses "axial compression" (higher span means smaller data size), *max_ring_diff* specifies the maximum ring difference to store, and *view_mash_factor* can be used to reduce the number of views (or azimutal angles).

Siemens uses span=1, max_ring_diff=60 and view_mash_factor=1. Here we will use a smaller data size to reduce computation time for the exercise. Feel free to change these numbers (if you know what you are doing...).

In [ ]:
template_acq_data = AcquisitionData('Siemens_mMR', span=11, max_ring_diff=15, view_mash_factor=2)
template_acq_data.write('template.hs')

In [ ]:
# create listmode-to-sinograms converter object
lm2sino = ListmodeToSinograms()

# set input, output and template files
lm2sino.set_input(list_file)
lm2sino.set_output_prefix(sino_file)
lm2sino.set_template('template.hs')

In [ ]:
# set timing interval (in secs) since start of acquisition
# (the listmode file provided is for 1 hour).
# you can vary this to see the effect on noise. Increasing it will mean somewhat longer
# processing time in the following steps (but not in the reconstruction).
lm2sino.set_time_interval(0, 600)  # 0 - 600 is the first 10 minutes
# set up the converter
lm2sino.set_up()
# create the prompts sinogram
lm2sino.process()

In [ ]:
# check the content of the directory. there should be a `sino*.hs`, `'.s` pair.
# The `.hs` file is an Interfile header pointing to the binary data.
%ls

## Check the prompts sinograms
The 3D PET data returned by `as_array` are organised by 2D sinogram. The exact order of the sinograms
is complicated for 3D PET, but they by *segment* (roughly: average ring difference). The first
segment corresponds to "segment 0", i.e. detector pairs which are (roughly) in the same 
detector ring. For a scanner with `N` rings, there will be `2N-1` (2D) sinograms in segment 0.

In [ ]:
# get access to the sinograms
acq_data = lm2sino.get_output()
# copy the acquisition data into a Python array
acq_array = acq_data.as_array()[0,:,:,:]  # first index is for ToF, which we don't have here
# how many counts total in this segment?
print('num prompts in this segments: %d' % acq_array.sum())
# print the data sizes. 
print('acquisition data dimensions: %dx%dx%d' % acq_array.shape)
# use a slice number for display that is appropriate for the NEMA phantom
z = 71
show_2D_array('Acquisition data', acq_array[z,:,:])

## Estimate the *randoms* background
Siemens stores *delayed coincidences*. These form a very noisy estimate of the
background due to accidental coincidences in the data. However, that estimate is too noisy
to be used in iterative image reconstruction.

SIRF uses an algorithm from STIR that gives a much less noisy estimate. The help message 
gives some information.

In [ ]:
help(lm2sino)

In [ ]:
# Get the randoms estimate
# This will take a while
randoms = lm2sino.estimate_randoms()

## Plot the randoms-estimate
A (2D) sinogram of the randoms has diagonal lines. This is related to the
detector efficiencies, but we cannot get into that here.

In [ ]:
randoms_array=randoms.as_array()[0,:,:,:]
show_2D_array('randoms', randoms_array[z,:,:])

# Reconstruct the data
We will reconstruct the data with increasingly accurate models for the acquisition as illustration.

For simplicity, we will use OSEM and use only a few sub-iterations for speed.

In [ ]:
# First just select an acquisition model that implements the geometric
# forward projection by a ray tracing matrix multiplication
acq_model = AcquisitionModelUsingRayTracingMatrix()
acq_model.set_num_tangential_LORs(10);

In [ ]:
# define objective function to be maximized as
# Poisson logarithmic likelihood (with linear model for mean)
obj_fun = make_Poisson_loglikelihood(acq_data)
obj_fun.set_acquisition_model(acq_model)

In [ ]:
# create the reconstruction object
recon = OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)

num_subsets = 7
# Feel free to increase these
num_subiterations = 4
recon.set_num_subsets(num_subsets)
recon.set_num_subiterations(num_subiterations)

In [ ]:
# create initial image estimate of dimensions and voxel sizes
# compatible with the scanner geometry (included in the AcquisitionData
# object acq_data) and initialize each voxel to 1.0
nxny = (127, 127)
initial_image = acq_data.create_uniform_image(1.0, nxny)

In [ ]:
image = initial_image
recon.set_up(image)
# set the initial image estimate
recon.set_current_estimate(image)
# reconstruct
recon.process()
# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Add detector sensitivity modelling
Each crystal pair will have different detection efficiency. We need to take that into account
in our acquisition model. The scanner provides a *normalisation file* to do this (the terminology
originates from the days that we were "normalising" by  dividing by the detected counts 
by the sensitivities.)

In SIRF, you can incorporate this effect in the acquisition model by using an `AcquisitionSensitivityModel`.

In [ ]:
# create it from the supplied file
asm_norm = AcquisitionSensitivityModel(norm_file)

In [ ]:
# add it to the acquisition model
acq_model.set_acquisition_sensitivity(asm_norm)

In [ ]:
# update the objective function
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
# reconstruct
image = initial_image
recon.set_up(image)
recon.set_current_estimate(image)
recon.process()
# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Add attenuation modeling

In [ ]:
# read attenuation image
attn_image = ImageData(attn_file)
z = 71
attn_image.show(z)

In [ ]:
attn_acq_model = AcquisitionModelUsingRayTracingMatrix()
asm_attn = AcquisitionSensitivityModel(attn_image, attn_acq_model)
# converting attenuation into attenuation factors (see previous exercise)
asm_attn.set_up(acq_data)
attn_factors = AcquisitionData(acq_data)
attn_factors.fill(1.0)
print('applying attenuation (please wait, may take a while)...')
asm_attn.unnormalise(attn_factors)

In [ ]:
# use these in the final attenuation model
asm_attn = AcquisitionSensitivityModel(attn_factors)

We now have two acquisition_sensitivity_models: for detection sensitivity and for
count loss due to attenuation. We combine them by "chaning" them together (which will
model the multiplication of both sensitivities).

In [ ]:
# chain attenuation and normalisation
asm = AcquisitionSensitivityModel(asm_norm, asm_attn)

In [ ]:
# update the acquisition model etc
acq_model.set_acquisition_sensitivity(asm)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
# reconstruct
image = initial_image
recon.set_up(image)
recon.set_current_estimate(image)
recon.process()
# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Add a background term for modelling the randoms

In [ ]:
acq_model.set_background_term(randoms)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
image = initial_image
recon.set_up(image)

In [ ]:
recon.set_current_estimate(image)
recon.process()
# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])